# 自适应检索增强型RAG系统
自适应检索(Adaptive Retrieval)系统可根据查询类型动态选择最优检索策略，该方法显著提升RAG系统在多样化问题场景下的响应准确性与相关性。

-----
不同查询类型需匹配差异化检索策略，本系统实现四阶优化流程：
1. 查询类型分类（事实性/分析性/意见性/上下文型）
2. 自适应选择检索策略
3. 执行专门的检索技术
4. 生成定制化响应

-----
实现步骤：
- 处理文档以提取文本，将其分块，并创建嵌入向量
- 对查询进行分类以确定其类型：查询分为事实性（Factual）、分析性（Analytical）、意见性（Opinion）或上下文相关性（Contextual）。
- 根据查询类型使用自适应检索策略检索文档
- 根据查询、检索到的文档和查询类型生成回答

In [1]:
import fitz
import os
import re
import json
import numpy as np
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

True

In [2]:
client = OpenAI(
    base_url=os.getenv("LLM_BASE_URL"),
    api_key=os.getenv("LLM_API_KEY")
)
llm_model = os.getenv("LLM_MODEL_ID")
embedding_model = os.getenv("EMBEDDING_MODEL_ID")

pdf_path = "../../data/AI_Information.en.zh-CN.pdf"

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本，并打印前 `num_chars` 个字符。

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # 打开 PDF 文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串以存储提取的文本

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")  # 从页面中提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

In [4]:
def chunk_text(text, n, overlap):
    """
    将文本分割为重叠的块

    Args:
    text (str): 要分割的文本
    n (int): 每个块的字符数
    overlap (int): 块之间的重叠字符数

    Returns:
    List[str]: 文本块列表
    """
    chunks = []  #
    for i in range(0, len(text), n - overlap):
        # 添加从当前索引到索引 + 块大小的文本块
        chunk = text[i:i + n]
        if chunk:
            chunks.append(chunk)

    return chunks  # Return the list of text chunks

In [5]:
class SimpleVectorStore:
    """
    使用NumPy实现的简单向量存储。
    """
    def __init__(self):
        """
        初始化向量存储。
        """
        self.vectors = []  # 用于存储嵌入向量的列表
        self.texts = []  # 用于存储原始文本的列表
        self.metadata = []  # 用于存储每个文本元数据的列表

    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。

        Args:
        text (str): 原始文本。
        embedding (List[float]): 嵌入向量。
        metadata (dict, 可选): 额外的元数据。
        """
        self.vectors.append(np.array(embedding))  # 将嵌入转换为numpy数组并添加到向量列表中
        self.texts.append(text)  # 将原始文本添加到文本列表中
        self.metadata.append(metadata or {})  # 添加元数据到元数据列表中，如果没有提供则使用空字典

    def similarity_search(self, query_embedding, k=5):
        """
        查找与查询嵌入最相似的项目。

        Args:
        query_embedding (List[float]): 查询嵌入向量。
        k (int): 返回的结果数量。

        Returns:
        List[Dict]: 包含文本和元数据的前k个最相似项。
        """
        if not self.vectors:
            return []  # 如果没有存储向量，则返回空列表

        # 将查询嵌入转换为numpy数组
        query_vector = np.array(query_embedding)

        # 使用余弦相似度计算相似度
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 计算查询向量与存储向量之间的余弦相似度
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 添加索引和相似度分数

        # 按相似度排序（降序）
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 返回前k个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 添加对应的文本
                "metadata": self.metadata[idx],  # 添加对应的元数据
                "similarity": score  # 添加相似度分数
            })

        return results  # 返回前k个最相似项的列表


In [6]:
def create_embeddings(text):
    """
    使用Embedding模型为给定文本创建嵌入向量。

    Args:
    text (str): 要创建嵌入向量的输入文本。

    Returns:
    List[float]: 嵌入向量。
    """
    # 通过将字符串输入转换为列表来处理字符串和列表输入
    input_text = text if isinstance(text, list) else [text]

    # 使用指定的模型为输入文本创建嵌入向量
    response = client.embeddings.create(
        model=embedding_model,
        input=input_text
    )

    # 如果输入是字符串，仅返回第一个嵌入向量
    if isinstance(text, str):
        return response.data[0].embedding

    # 否则，将所有嵌入向量作为向量列表返回
    return [item.embedding for item in response.data]

In [7]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    为RAG处理文档。

    Args:
    pdf_path (str): PDF文件的路径。
    chunk_size (int): 每个文本块的大小（以字符为单位）。
    chunk_overlap (int): 文本块之间的重叠大小（以字符为单位）。

    Returns:
    Tuple[List[str], SimpleVectorStore]: 包含文档文本块及其嵌入向量的向量存储。
    """
    print("从PDF中提取文本...")
    extracted_text = extract_text_from_pdf(pdf_path)  # 调用函数提取PDF中的文本

    print("分割文本...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)  # 将提取的文本分割为多个块
    print(f"创建了 {len(chunks)} 个文本块")

    print("为文本块创建嵌入向量...")
    # 为了提高效率，一次性为所有文本块创建嵌入向量
    chunk_embeddings = create_embeddings(chunks)

    # 创建向量存储
    store = SimpleVectorStore()

    # 将文本块添加到向量存储中
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,  # 文本内容
            embedding=embedding,  # 嵌入向量
            metadata={"index": i, "source": pdf_path}  # 元数据，包括索引和源文件路径
        )

    print(f"向向量存储中添加了 {len(chunks)} 个文本块")
    return chunks, store

## 查询分类

In [8]:
def classify_query(query):
    """
    将查询分类为四个类别之一：事实性（Factual）、分析性（Analytical）、意见性（Opinion）或上下文相关性（Contextual）。

    Args:
        query (str): 用户查询

    Returns:
        str: 查询类别
    """
    # 定义系统提示以指导AI进行分类
    system_prompt = """您是专业的查询分类专家。
        请将给定查询严格分类至以下四类中的唯一一项：
        - Factual：需要具体、可验证信息的查询
        - Analytical：需要综合分析或深入解释的查询
        - Opinion：涉及主观问题或寻求多元观点的查询
        - Contextual：依赖用户具体情境的查询

        请仅返回分类名称，不要添加任何解释或额外文本。
    """

    # 创建包含要分类查询的用户提示
    user_prompt = f"对以下查询进行分类: {query}"

    # 从AI模型生成分类响应
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 从响应中提取并去除多余的空白字符以获取类别
    category = response.choices[0].message.content.strip()

    # 定义有效的类别列表
    valid_categories = ["Factual", "Analytical", "Opinion", "Contextual"]

    # 确保返回的类别是有效的
    for valid in valid_categories:
        if valid in category:
            return valid

    # 如果分类失败，默认返回“Factual”（事实性）
    return "Factual"


## 专项检索策略实现方案
### 1. 事实性策略 - 精准导向

In [9]:
def factual_retrieval_strategy(query, vector_store, k=4):
    """
    针对事实性查询的检索策略，专注于精确度。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储库
        k (int): 返回的文档数量

    Returns:
        List[Dict]: 检索到的文档列表
    """
    print(f"执行事实性检索策略: '{query}'")

    # 使用LLM增强查询以提高精确度
    system_prompt = """您是搜索查询优化专家。
        您的任务是重构给定的事实性查询，使其更精确具体以提升信息检索效果。
        重点关注关键实体及其关联关系。

        请仅提供优化后的查询，不要包含任何解释。
    """

    user_prompt = f"请优化此事实性查询: {query}"

    # 使用LLM生成增强后的查询
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 提取并打印增强后的查询
    enhanced_query = response.choices[0].message.content.strip()
    print(f"优化后的查询: {enhanced_query}")

    # 为增强后的查询创建嵌入向量
    query_embedding = create_embeddings(enhanced_query)

    # 执行初始相似性搜索以检索文档
    initial_results = vector_store.similarity_search(query_embedding, k=k*2)

    # 初始化一个列表来存储排序后的结果
    ranked_results = []

    # 使用LLM对文档进行评分和排序
    for doc in initial_results:
        relevance_score = score_document_relevance(enhanced_query, doc["text"])
        ranked_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "similarity": doc["similarity"],
            "relevance_score": relevance_score
        })

    # 按相关性得分降序排列结果
    ranked_results.sort(key=lambda x: x["relevance_score"], reverse=True)

    # 返回前k个结果
    return ranked_results[:k]


### 2. 分析性策略 - 全面覆盖


In [10]:
def analytical_retrieval_strategy(query, vector_store, k=4):
    """
    针对分析性查询的检索策略，专注于全面覆盖。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储库
        k (int): 返回的文档数量

    Returns:
        List[Dict]: 检索到的文档列表
    """
    print(f"执行分析性检索策略: '{query}'")

    # 定义系统提示以指导AI生成子问题
    system_prompt = """您是复杂问题拆解专家。
    请针对给定的分析性查询生成探索不同维度的子问题。
    这些子问题应覆盖主题的广度并帮助获取全面信息。

    请严格生成恰好3个子问题，每个问题单独一行。
    """

    # 创建包含主查询的用户提示
    user_prompt = f"请为此分析性查询生成子问题：{query}"

    # 使用LLM生成子问题
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )

    # 提取并清理子问题
    sub_queries = response.choices[0].message.content.strip().split('\n')
    sub_queries = [q.strip() for q in sub_queries if q.strip()]
    print(f"生成的子问题: {sub_queries}")

    # 为每个子问题检索文档
    all_results = []
    for sub_query in sub_queries:
        # 为子问题创建嵌入向量
        sub_query_embedding = create_embeddings(sub_query)
        # 执行相似性搜索以获取子问题的结果
        results = vector_store.similarity_search(sub_query_embedding, k=2)
        all_results.extend(results)

    # 确保多样性，从不同的子问题结果中选择
    # 移除重复项（相同的文本内容）
    unique_texts = set()
    diverse_results = []

    for result in all_results:
        if result["text"] not in unique_texts:
            unique_texts.add(result["text"])
            diverse_results.append(result)

    # 如果需要更多结果以达到k，则从初始结果中添加更多
    if len(diverse_results) < k:
        # 对主查询直接检索
        main_query_embedding = create_embeddings(query)
        main_results = vector_store.similarity_search(main_query_embedding, k=k)

        for result in main_results:
            if result["text"] not in unique_texts and len(diverse_results) < k:
                unique_texts.add(result["text"])
                diverse_results.append(result)

    # 返回前k个多样化的结果
    return diverse_results[:k]


### 3. 观点性策略 - 多元视角


In [11]:
def opinion_retrieval_strategy(query, vector_store, k=4):
    """
    针对观点查询的检索策略，专注于多样化的观点。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储库
        k (int): 返回的文档数量

    Returns:
        List[Dict]: 检索到的文档列表
    """
    print(f"执行观点检索策略: '{query}'")

    # 定义系统提示以指导AI识别不同观点
    system_prompt = """您是主题多视角分析专家。
        针对给定的观点类或意见类查询，请识别人们可能持有的不同立场或观点。

        请严格返回恰好3个不同观点角度，每个角度单独一行。
    """

    # 创建包含主查询的用户提示
    user_prompt = f"请识别以下主题的不同观点：{query}"

    # 使用LLM生成不同的观点
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )

    # 提取并清理观点
    viewpoints = response.choices[0].message.content.strip().split('\n')
    viewpoints = [v.strip() for v in viewpoints if v.strip()]
    print(f"已识别的观点: {viewpoints}")

    # 检索代表每个观点的文档
    all_results = []
    for viewpoint in viewpoints:
        # 将主查询与观点结合
        combined_query = f"{query} {viewpoint}"
        # 为组合查询创建嵌入向量
        viewpoint_embedding = create_embeddings(combined_query)
        # 执行相似性搜索以获取组合查询的结果
        results = vector_store.similarity_search(viewpoint_embedding, k=2)

        # 标记结果所代表的观点
        for result in results:
            result["viewpoint"] = viewpoint

        # 将结果添加到所有结果列表中
        all_results.extend(results)

    # 选择多样化的意见范围
    # 尽量确保从每个观点中至少获得一个文档
    selected_results = []
    for viewpoint in viewpoints:
        # 按观点过滤文档
        viewpoint_docs = [r for r in all_results if r.get("viewpoint") == viewpoint]
        if viewpoint_docs:
            selected_results.append(viewpoint_docs[0])

    # 用最高相似度的文档填充剩余的槽位
    remaining_slots = k - len(selected_results)
    if remaining_slots > 0:
        # 按相似度排序剩余文档
        remaining_docs = [r for r in all_results if r not in selected_results]
        remaining_docs.sort(key=lambda x: x["similarity"], reverse=True)
        selected_results.extend(remaining_docs[:remaining_slots])

    # 返回前k个结果
    return selected_results[:k]


### 4. 上下文型策略 - 情境整合


In [12]:
def contextual_retrieval_strategy(query, vector_store, k=4, user_context=None):
    """
    针对上下文查询的检索策略，结合用户提供的上下文信息。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储库
        k (int): 返回的文档数量
        user_context (str): 额外的用户上下文信息

    Returns:
        List[Dict]: 检索到的文档列表
    """
    print(f"执行上下文检索策略: '{query}'")

    # 如果未提供用户上下文，则尝试从查询中推断上下文
    if not user_context:
        system_prompt = """您是理解查询隐含上下文的专家。
        对于给定的查询，请推断可能相关或隐含但未明确说明的上下文信息。
        重点关注有助于回答该查询的背景信息。

        请简要描述推断的隐含上下文。
        """

        user_prompt = f"推断此查询中的隐含背景(上下文)：{query}"

        # 使用LLM生成推断出的上下文
        response = client.chat.completions.create(
            model=llm_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1
        )

        # 提取并打印推断出的上下文
        user_context = response.choices[0].message.content.strip()
        print(f"推断出的上下文: {user_context}")

    # 重新表述查询以结合上下文
    system_prompt = """您是上下文整合式查询重构专家。
    根据提供的查询和上下文信息，请重新构建更具体的查询以整合上下文，从而获取更相关的信息。

    请仅返回重新构建的查询，不要包含任何解释。
    """

    user_prompt = f"""
    原始查询：{query}
    关联上下文：{user_context}

    请结合此上下文重新构建查询：
    """

    # 使用LLM生成结合上下文的查询
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 提取并打印结合上下文的查询
    contextualized_query = response.choices[0].message.content.strip()
    print(f"结合上下文的查询: {contextualized_query}")

    # 基于结合上下文的查询检索文档
    query_embedding = create_embeddings(contextualized_query)
    initial_results = vector_store.similarity_search(query_embedding, k=k*2)

    # 根据相关性和用户上下文对文档进行排序
    ranked_results = []

    for doc in initial_results:
        # 计算文档在考虑上下文情况下的相关性得分
        context_relevance = score_document_context_relevance(query, user_context, doc["text"])
        ranked_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "similarity": doc["similarity"],
            "context_relevance": context_relevance
        })

    # 按上下文相关性排序，并返回前k个结果
    ranked_results.sort(key=lambda x: x["context_relevance"], reverse=True)
    return ranked_results[:k]


## 文档评分辅助函数

In [13]:
def score_document_relevance(query, document):
    """
    使用LLM对文档与查询的相关性进行评分。

    Args:
        query (str): 用户查询
        document (str): 文档文本

    Returns:
        float: 相关性评分，范围为0-10
    """
    # 系统提示，指导模型如何评估相关性
    system_prompt = """您是文档相关性评估专家。
        请根据文档与查询的匹配程度给出0到10分的评分：
        0 = 完全无关
        10 = 完美契合查询

        请仅返回一个0到10之间的数字评分，不要包含任何其他内容。
    """

    # 如果文档过长，则截断文档
    doc_preview = document[:1500] + "..." if len(document) > 1500 else document

    # 包含查询和文档预览的用户提示
    user_prompt = f"""
        查询: {query}

        文档: {doc_preview}

        相关性评分（0-10）：
    """

    # 使用模型生成响应
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 从模型的响应中提取评分
    score_text = response.choices[0].message.content.strip()

    # 使用正则表达式提取数值评分
    match = re.search(r'(\d+(\.\d+)?)', score_text)
    if match:
        score = float(match.group(1))
        return min(10, max(0, score))  # 确保评分在0-10范围内
    else:
        # 如果提取失败，则返回默认评分
        return 5.0


In [14]:
def score_document_context_relevance(query, context, document):
    """
    根据查询和上下文评估文档的相关性。

    Args:
        query (str): 用户查询
        context (str): 用户上下文
        document (str): 文档文本

    Returns:
        float: 相关性评分，范围为0-10
    """
    # 系统提示，指导模型如何根据上下文评估相关性
    system_prompt = """您是结合上下文评估文档相关性的专家。
        请根据文档在给定上下文中对查询的响应质量，给出0到10分的评分：
        0 = 完全无关
        10 = 在给定上下文中完美契合查询

        请严格仅返回一个0到10之间的数字评分，不要包含任何其他内容。
    """

    # 如果文档过长，则截断文档
    doc_preview = document[:1500] + "..." if len(document) > 1500 else document

    # 包含查询、上下文和文档预览的用户提示
    user_prompt = f"""
    待评估查询：{query}
    关联上下文：{context}

    文档内容预览：
    {doc_preview}

    结合上下文的相关性评分（0-10）：
    """

    # 使用模型生成响应
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 从模型的响应中提取评分
    score_text = response.choices[0].message.content.strip()

    # 使用正则表达式提取数值评分
    match = re.search(r'(\d+(\.\d+)?)', score_text)
    if match:
        score = float(match.group(1))
        return min(10, max(0, score))  # 确保评分在0-10范围内
    else:
        # 如果提取失败，则返回默认评分
        return 5.0


## 自适应检索的核心函数

In [15]:
def adaptive_retrieval(query, vector_store, k=4, user_context=None):
    """
    执行自适应检索，通过选择并执行适当的检索策略。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储
        k (int): 要检索的文档数量
        user_context (str): 可选的用户上下文，用于上下文相关的查询

    Returns:
        List[Dict]: 检索到的文档列表
    """
    # 对查询进行分类以确定其类型
    query_type = classify_query(query)
    print(f"查询被分类为: {query_type}")

    # 根据查询类型选择并执行适当的检索策略
    if query_type == "Factual":
        # 使用事实检索策略获取精确信息
        results = factual_retrieval_strategy(query, vector_store, k)
    elif query_type == "Analytical":
        # 使用分析检索策略实现全面覆盖
        results = analytical_retrieval_strategy(query, vector_store, k)
    elif query_type == "Opinion":
        # 使用观点检索策略获取多样化的观点
        results = opinion_retrieval_strategy(query, vector_store, k)
    elif query_type == "Contextual":
        # 使用上下文检索策略，并结合用户上下文
        results = contextual_retrieval_strategy(query, vector_store, k, user_context)
    else:
        # 如果分类失败，默认使用事实检索策略
        results = factual_retrieval_strategy(query, vector_store, k)

    return results  # 返回检索到的文档


## 回答生成

In [16]:
def generate_response(query, results, query_type):
    """
    根据查询、检索到的文档和查询类型生成响应。

    Args:
        query (str): 用户查询
        results (List[Dict]): 检索到的文档列表
        query_type (str): 查询类型

    Returns:
        str: 生成的响应
    """
    # 从检索到的文档中准备上下文，通过连接它们的文本并使用分隔符
    context = "\n\n---\n\n".join([r["text"] for r in results])

    # 根据查询类型创建自定义系统提示
    if query_type == "Factual":
        system_prompt = """您是基于事实信息应答的AI助手。
    请严格根据提供的上下文回答问题，确保信息准确无误。
    若上下文缺乏必要信息，请明确指出信息局限。"""

    elif query_type == "Analytical":
        system_prompt = """您是专业分析型AI助手。
    请基于提供的上下文，对主题进行多维度深度解析：
    - 涵盖不同层面的关键要素（不同方面和视角）
    - 整合多方观点形成系统分析
    若上下文存在信息缺口或空白，请在分析时明确指出信息短缺。"""

    elif query_type == "Opinion":
        system_prompt = """您是观点整合型AI助手。
    请基于提供的上下文，结合以下标准给出不同观点：
    - 全面呈现不同立场观点
    - 保持各观点表述的中立平衡，避免出现偏见
    - 当上下文视角有限时，直接说明"""

    elif query_type == "Contextual":
        system_prompt = """您是情境上下文感知型AI助手。
    请结合查询背景与上下文信息：
    - 建立问题情境与文档内容的关联
    - 当上下文无法完全匹配具体情境时，请明确说明适配性限制"""

    else:
        system_prompt = """您是通用型AI助手。请基于上下文回答问题，若信息不足请明确说明。"""

    # 通过结合上下文和查询创建用户提示
    user_prompt = f"""
    上下文:
    {context}

    问题: {query}

    请基于上下文提供专业可靠的回答。
    """

    # 使用 OpenAI 生成响应
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    # 返回生成的响应内容
    return response.choices[0].message.content


## 完整的自适应检索 RAG 流程

In [17]:
def rag_with_adaptive_retrieval(pdf_path, query, k=4, user_context=None):
    """
    完整的RAG管道，带有自适应检索功能。

    Args:
        pdf_path (str): PDF文档的路径
        query (str): 用户查询
        k (int): 要检索的文档数量
        user_context (str): 可选的用户上下文

    Returns:
        Dict: 包含查询、检索到的文档、查询类型和响应的结果字典
    """
    print("\n=== RAG WITH ADAPTIVE RETRIEVAL ===")
    print(f"Query: {query}")  # 打印查询内容

    # 处理文档以提取文本，将其分块，并创建嵌入向量
    chunks, vector_store = process_document(pdf_path)

    # 对查询进行分类以确定其类型
    query_type = classify_query(query)
    print(f"Query classified as: {query_type}")  # 打印查询被分类为的类型

    # 根据查询类型使用自适应检索策略检索文档
    retrieved_docs = adaptive_retrieval(query, vector_store, k, user_context)

    # 根据查询、检索到的文档和查询类型生成响应
    response = generate_response(query, retrieved_docs, query_type)

    # 将结果编译成一个字典
    result = {
        "query": query,  # 用户查询
        "query_type": query_type,  # 查询类型
        "retrieved_documents": retrieved_docs,  # 检索到的文档
        "response": response  # 生成的响应
    }

    print("\n=== RESPONSE ===")  # 打印响应标题
    print(response)  # 打印生成的响应

    return result  # 返回结果字典


## 评价框架


In [18]:
def evaluate_adaptive_vs_standard(pdf_path, test_queries, reference_answers=None):
    """
    对比分析自适应检索与标准检索在测试查询集上的表现。

    本函数实现以下评估流程：
    1. 文档预处理与分块，构建向量存储
    2. 并行执行标准检索与自适应检索
    3. 双通道结果对比分析
    4. 若存在参考答案，执行回答质量评估

    Args:
        pdf_path (str): 作为知识源的PDF文档路径
        test_queries (List[str]): 用于评估两种检索方法的测试查询列表
        reference_answers (List[str], 可选): 用于评估指标的参考答案列表

    Returns:
        Dict: 包含每个查询的单独结果和整体比较的评估结果
    """
    print("=== 正在评估自适应检索与标准检索 ===")

    # 处理文档以提取文本，创建分块并构建向量存储
    chunks, vector_store = process_document(pdf_path)

    # 初始化用于存储比较结果的集合
    results = []

    # 对每个测试查询使用两种检索方法进行处理
    for i, query in enumerate(test_queries):
        print(f"\n\n查询 {i+1}: {query}")

        # --- 标准检索方法 ---
        print("\n--- 标准检索 ---")
        # 为查询创建嵌入向量
        query_embedding = create_embeddings(query)
        # 使用简单的向量相似性检索文档
        standard_docs = vector_store.similarity_search(query_embedding, k=4)
        # 使用通用方法生成响应
        standard_response = generate_response(query, standard_docs, "General")

        # --- 自适应检索方法 ---
        print("\n--- 自适应检索 ---")
        # 对查询进行分类以确定其类型（事实型、分析型、意见型、上下文型）
        query_type = classify_query(query)
        # 使用适合此查询类型的策略检索文档
        adaptive_docs = adaptive_retrieval(query, vector_store, k=4)
        # 根据查询类型生成定制化的响应
        adaptive_response = generate_response(query, adaptive_docs, query_type)

        # 存储此查询的完整结果
        result = {
            "query": query,  # 查询内容
            "query_type": query_type,  # 查询类型
            "standard_retrieval": {  # 标准检索结果
                "documents": standard_docs,  # 检索到的文档
                "response": standard_response  # 生成的响应
            },
            "adaptive_retrieval": {  # 自适应检索结果
                "documents": adaptive_docs,  # 检索到的文档
                "response": adaptive_response  # 生成的响应
            }
        }

        # 如果有可用的参考答案，则添加到结果中
        if reference_answers and i < len(reference_answers):
            result["reference_answer"] = reference_answers[i]

        results.append(result)  # 将结果添加到结果列表中

        # 显示两种响应的简要预览以便快速比较
        print("\n--- 响应 ---")
        print(f"标准: {standard_response[:200]}...")  # 标准检索的响应前200个字符
        print(f"自适应: {adaptive_response[:200]}...")  # 自适应检索的响应前200个字符

    # 如果有参考答案，则计算比较指标
    if reference_answers:
        comparison = compare_responses(results)  # 调用 compare_responses 函数进行详细比较
        print("\n=== 评估结果 ===")
        print(comparison)  # 打印比较结果

    # 返回完整的评估结果
    return {
        "results": results,  # 每个查询的详细结果
        "comparison": comparison if reference_answers else "未提供参考答案以进行评估"  # 总体比较结果或提示信息
    }


In [19]:
def compare_responses(results):
    """
    比较标准检索、自适应检索的响应与参考答案。

    Args:
        results (List[Dict]): 包含两种类型响应的结果列表

    Returns:
        str: 比较分析结果
    """
    # 定义系统提示，指导AI如何比较响应
    comparison_prompt = """您是信息检索系统评估专家。
    请对比分析标准检索与自适应检索对每个查询的响应质量。
    评估维度需包含：
    - 信息准确性
    - 内容相关性
    - 回答全面性
    - 与参考答案的契合度（一致性）

    要求提供每种方法的优势与不足的详细分析报告。"""

    # 初始化对比文本的标题
    comparison_text = "# 标准检索与自适应检索效果评估\n\n"

    # 遍历每个结果以比较响应
    for i, result in enumerate(results):
        # 如果查询没有参考答案，则跳过
        if "reference_answer" not in result:
            continue

        # 将查询详情添加到比较文本中
        comparison_text += f"## 查询 {i+1}: {result['query']}\n"
        comparison_text += f"*查询类型: {result['query_type']}*\n\n"
        comparison_text += f"**参考答案:**\n{result['reference_answer']}\n\n"

        # 将标准检索响应添加到比较文本中
        comparison_text += f"**标准检索响应:**\n{result['standard_retrieval']['response']}\n\n"

        # 将自适应检索响应添加到比较文本中
        comparison_text += f"**自适应检索响应:**\n{result['adaptive_retrieval']['response']}\n\n"

        # 创建用户提示，让AI比较这两种响应
        user_prompt = f"""
        参考答案：{result['reference_answer']}

        标准检索响应：{result['standard_retrieval']['response']}

        自适应检索响应：{result['adaptive_retrieval']['response']}

        请从以下维度进行详细对比分析：
        1. 核心事实的准确度差异
        2. 上下文关联强度对比
        3. 信息完整度评估
        4. 与参考答案的语义契合度
        """

        # 使用OpenAI客户端生成比较分析
        response = client.chat.completions.create(
            model=llm_model,
            messages=[
                {"role": "system", "content": comparison_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2
        )

        # 将AI的比较分析结果添加到比较文本中
        comparison_text += f"**比较分析:**\n{response.choices[0].message.content}\n\n"

    return comparison_text  # 返回完整的比较分析文本


## 评估自适应检索系统（自定义查询）

In [20]:
# 定义涵盖不同查询类型的测试查询，以展示自适应检索如何处理各种查询意图
test_queries = [
    "什么是可解释的人工智能（XAI）？",           # 事实性查询 - 寻求定义/具体信息
    # "AI伦理和治理框架如何应对潜在的社会影响？",  # 分析性查询 - 需要全面分析
    # "AI发展是否过快以至于无法进行适当监管？",    # 意见性查询 - 寻求多样化观点
    # "可解释的AI如何帮助医疗决策？",            # 上下文感知查询 - 从上下文中受益
]

# 更全面评估的参考答案
# 这些可以用来客观地评估响应质量，与已知标准进行对比
reference_answers = [
    "可解释的人工智能（XAI）旨在通过提供清晰的决策过程解释，使AI系统变得透明且易于理解。这有助于用户信任并有效管理AI技术。",
    # "AI伦理和治理框架通过制定指南和原则来确保AI系统的负责任开发和使用，以应对潜在的社会影响。这些框架关注公平性、问责制、透明度以及保护人权，从而降低风险并促进有益的产出。",
    # "关于AI发展是否过快而无法进行适当监管，意见不一。有些人认为快速进步超出了监管努力，可能导致潜在风险和伦理问题。另一些人则认为创新应保持当前速度，同时让法规与时俱进以应对新兴挑战。",
    # "可解释的AI可以通过提供对AI驱动建议的透明且易于理解的见解，显著帮助医疗决策。这种透明性有助于医疗专业人员信任AI系统，做出明智决策，并通过理解AI建议背后的理由来改善患者结果。"
]


In [21]:
# 运行评估以比较自适应检索与标准检索
# 这将使用两种方法处理每个查询并比较结果
evaluation_results = evaluate_adaptive_vs_standard(
    pdf_path=pdf_path,                  # 用于知识提取的源文档
    test_queries=test_queries,          # 要评估的测试查询列表
    reference_answers=reference_answers  # 可选的参考答案用于对比
)

print(evaluation_results)

# 结果将显示标准检索和自适应检索在不同查询类型下的详细对比，
# 突出显示自适应策略提供改进结果的地方
print("评估结果".center(80, "="))
print(evaluation_results["comparison"])


=== 正在评估自适应检索与标准检索 ===
从PDF中提取文本...
分割文本...
创建了 13 个文本块
为文本块创建嵌入向量...
向向量存储中添加了 13 个文本块


查询 1: 什么是可解释的人工智能（XAI）？

--- 标准检索 ---

--- 自适应检索 ---
查询被分类为: Analytical
执行分析性检索策略: '什么是可解释的人工智能（XAI）？'
生成的子问题: ['1. 可解释的人工智能（XAI）与传统黑盒AI模型相比有哪些主要区别和优势？', '2. 可解释的人工智能（XAI）在实际应用中有哪些常见的技术和方法？', '3. 可解释的人工智能（XAI）在医疗、金融等关键领域如何帮助提高透明度和可信度？']

--- 响应 ---
标准: 可解释人工智能（XAI，Explainable AI）是人工智能领域的一个重要分支，旨在通过增强AI系统的透明性和可理解性，使人类能够理解其决策逻辑和内部运作机制。根据上下文，以下是关于XAI的专业解析：

### 核心定义与目标
XAI通过技术手段揭示传统"黑箱"模型（如深度学习）的决策过程，解决因模型复杂性导致的信任和问责问题。其核心目标是：
1. **透明度**：展示模型如何从输入数据推导出...
自适应: ### 可解释人工智能（XAI）的多维度深度解析

#### 1. **核心定义与目标**
可解释人工智能（XAI, Explainable Artificial Intelligence）是一类旨在提升AI系统透明度和决策可理解性的技术框架。其核心目标是解决传统AI（尤其是深度学习）的"黑箱问题"，通过揭示模型的内部逻辑，使用户能够理解：
- **决策依据**：模型基于哪些输入特征或数据模式做出...

=== 评估结果 ===
# 标准检索与自适应检索效果评估

## 查询 1: 什么是可解释的人工智能（XAI）？
*查询类型: Analytical*

**参考答案:**
可解释的人工智能（XAI）旨在通过提供清晰的决策过程解释，使AI系统变得透明且易于理解。这有助于用户信任并有效管理AI技术。

**标准检索响应:**
可解释人工智能（XAI，Explainable AI）是人工智能领域的一个重要分支，旨在通过增强AI系统的透明性和可理解性，使人类能够理解其决策逻辑和内部运作